In [ ]:
import json
import os
import sys

from sentence_transformers import SentenceTransformer
import torch
import torchvision.datasets as dset

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from testing import *

In [ ]:
available_models = [
    "all-mpnet-base-v2",
    "multi-qa-mpnet-base-dot-v1",
    "all-distilroberta-v1",
    "all-MiniLM-L12-v2",
    "multi-qa-distilbert-cos-v1",
    "all-MiniLM-L6-v2",
    "multi-qa-MiniLM-L6-cos-v1",
    "paraphrase-multilingual-mpnet-base-v2",
    "paraphrase-albert-small-v2",
    "paraphrase-multilingual-MiniLM-L12-v2",
    "paraphrase-MiniLM-L3-v2",
    "distiluse-base-multilingual-cased-v1",
    "distiluse-base-multilingual-cased-v2"
]

scores = {model: {"similarity_score": 0, "dissimilarity_score": 0} for model in available_models}

In [ ]:
device = torch.device("cuda:3")

ds = dset.CocoCaptions(
    root = '/home/xbuban1/coco/images/val2017',
    annFile = '/home/xbuban1/coco/annotations/captions_val2017.json'
)

for i, model_name in enumerate(available_models):
    print(f"Processing model {model_name} ({i+1}/{len(available_models)})")
    model = SentenceTransformer(model_name, device=device)

    ds_s = get_intra_similarities_avg(model, ds)
    ds_d = get_intra_dissimilarities_avg(model, ds)

    scores[model_name]["similarity_score"] = ds_s
    scores[model_name]["dissimilarity_score"] = ds_d

    print()

In [ ]:
# Order print by similarity score
for model_name in sorted(scores, key=lambda x: scores[x]["similarity_score"], reverse=True):
    print(f"{model_name.ljust(40)} {scores[model_name]['similarity_score']:0.4f}")

In [ ]:
# Order print by dissimilarity score
for model_name in sorted(scores, key=lambda x: scores[x]["dissimilarity_score"], reverse=False):
    print(f"{model_name.ljust(40)} {scores[model_name]['dissimilarity_score']:0.4f}")

In [ ]:
# Order print by combined score
for model_name in sorted(scores, key=lambda x: scores[x]["similarity_score"] - scores[x]["dissimilarity_score"], reverse=True):
    print(f"{model_name.ljust(40)} {scores[model_name]['similarity_score']:0.4f} | {scores[model_name]['dissimilarity_score']:0.4f}")

In [ ]:
model_name = "paraphrase-multilingual-MiniLM-L12-v2"

with open('captions/val_224_no_merge.json') as f:
    captions = json.load(f)

with open('captions/ved.json') as f:
    ved_captions = json.load(f)

model = SentenceTransformer(model_name, device=device)

ds_similarities = get_intra_similarities(model, ds)
ds_avg_sim = torch.cat(ds_similarities).mean(dim=0)
ds_similarities = [sim.mean() for sim in ds_similarities]

ved_similarities = get_similarities(model, ds, ved_captions)
ved_avg_sim = torch.cat(ved_similarities).mean(dim=0)
ved_similarities = [sim.mean() for sim in ved_similarities]

similarities = get_similarities(model, ds, captions)
avg_sim = torch.cat(similarities).mean(dim=0)
similarities = [sim.mean() for sim in similarities]

print(f"Intra Captions similarity score: {ds_avg_sim:0.4f}")
print(f"VED Captions similarity score: {ved_avg_sim:0.4f}")
print(f"Captions similarity score: {avg_sim:0.4f}")

In [ ]:
boxplot_similarities(ds_similarities, ved_similarities, similarities, ds_avg_sim, ved_avg_sim, avg_sim)